In [1]:
#TO DO

# 1. DONE Make number of slots a variable

# 2. Look at mapping the optimal slots onto a timetable

# 3. Test google form data template and edit data ingress to check - 
#    if we could build and do this we can see if people in the room 
# https://docs.google.com/forms/d/1eou9g1pQeVUu2bG3oOVr-EcGuSQQ953Vcb3-fpBC9N4/edit

# 4. Could you add extra constraints e.g. if all people studying x also study y (even if x is small) 
# don't group them together because it essentially creates another exam slot from the double booking. *started with dict of number of people studying each subject

# 5. DONE A function that gives the lowest number of slots required to achieve a certain number of clashes 
# or the lowest number of slots required to achieve a certain percentage of clashes in terms of the total number of exams being taken.

# 6. Get the function to flag any triple clashes (needs to revert back to check slots against original dataframe)

In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
import math
from collections import defaultdict
import os

In [3]:
# Variables:
NUMBER_OF_SLOTS = 3
ACCEPTABLE_CLASHES = 2
ACCEPTABLE_CLASH_PERCENTAGE = 10

In [4]:
# Manually read in data file

file_path = os.path.join('data', 'test_data2.csv')

# Read the file into a pandas DataFrame

df_raw = pd.read_csv(file_path)

# Drop subject column
#df_raw = df_raw.drop(columns="Subject")
df_raw = df_raw.drop(df_raw.columns[0], axis=1)
df_raw =  df_raw.fillna(0)

# Will probably need to replace "Yes" with 1 and "No" with 0.

# Most subjects for one student
subjects_studied = []
for i in range(0,len(df_raw)):
    count = np.count_nonzero(df_raw.iloc[i]==1)
    subjects_studied.append(count)
    
Most_subjects = max(subjects_studied)
Total_exams = sum(subjects_studied)
Minimum_slots = round(Most_subjects/1.999) #bad solution

print(df_raw.head())
print(f"The most number of subjects studied by one student is {Most_subjects}, therefore to avoid triple clashes you will required at least {Minimum_slots} slots")
print(f"The total number of subjects studied for all students is {Total_exams}")

   Maths  Physics  Chemistry  Philosophy  Further Maths  German  Biology  \
0      1        1          0           1              1       0        0   
1      1        1          0           0              1       1        0   
2      1        1          1           0              1       0        0   
3      1        0          1           0              1       0        0   
4      1        0          1           0              0       0        1   

   Computer Science  Economics  OTHER  
0                 0          0      0  
1                 0          0      0  
2                 0          0      0  
3                 0          0      0  
4                 0          0      0  
The most number of subjects studied by one student is 5, therefore to avoid triple clashes you will required at least 3 slots
The total number of subjects studied for all students is 76


In [5]:
# Create dict of total students studying each subject - 
# Can be used to not include combinations where all students from a subject have a clash
subs = df_raw.columns.tolist()
students_numbers = []
for i in range(0,len(subs)):
    count = np.count_nonzero(df_raw.loc[i]==1)  
    students_numbers.append(count)
per_subject_dict = dict(zip(subs, students_numbers))
print (per_subject_dict)

{'Maths': 4, 'Physics': 4, 'Chemistry': 4, 'Philosophy': 3, 'Further Maths': 3, 'German': 4, 'Biology': 5, 'Computer Science': 3, 'Economics': 3, 'OTHER': 5}


In [6]:
# Number of subjects
number_of_subs = len(df_raw.columns)
# Create subject list 1

subject_1_list = []
for n in range(0, number_of_subs-1):
    for i in range (1, number_of_subs-n):
        subject_1_list.append(df_raw.columns[n])
print(subject_1_list)

['Maths', 'Maths', 'Maths', 'Maths', 'Maths', 'Maths', 'Maths', 'Maths', 'Maths', 'Physics', 'Physics', 'Physics', 'Physics', 'Physics', 'Physics', 'Physics', 'Physics', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Philosophy', 'Philosophy', 'Philosophy', 'Philosophy', 'Philosophy', 'Philosophy', 'Further Maths', 'Further Maths', 'Further Maths', 'Further Maths', 'Further Maths', 'German', 'German', 'German', 'German', 'Biology', 'Biology', 'Biology', 'Computer Science', 'Computer Science', 'Economics']


In [7]:
# Create subject list 2
subject_2_list = []
for n in range(0, number_of_subs-1):
    for i in range (1, number_of_subs-n):
        subject_2_list.append(df_raw.columns[i+n])
print(subject_2_list)

if len(subject_1_list) == len(subject_2_list):
    print("List are the same length")
else:
    print("Lists are different lengths, PLEASE CHECK")

['Physics', 'Chemistry', 'Philosophy', 'Further Maths', 'German', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'Chemistry', 'Philosophy', 'Further Maths', 'German', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'Philosophy', 'Further Maths', 'German', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'Further Maths', 'German', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'German', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'Biology', 'Computer Science', 'Economics', 'OTHER', 'Computer Science', 'Economics', 'OTHER', 'Economics', 'OTHER', 'OTHER']
List are the same length


In [8]:
# Build clashes data

Clashes = []

for i in range(0, len(subject_1_list)):
    Count = 0
    for n in range(0, len(df_raw)):
        if df_raw[subject_1_list[i]].iloc[n] == 1 and df_raw[subject_2_list[i]].iloc[n]:
            Count += 1
    Clashes.append(Count)
    
print(Clashes)
        

[8, 7, 2, 11, 1, 4, 6, 4, 5, 3, 2, 5, 1, 1, 1, 0, 3, 0, 3, 0, 2, 1, 1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 3, 2, 3, 0, 0, 0, 0, 3, 1, 1, 0, 2, 1]


In [9]:
# We list all pairs of subjects twice (in both orders)
df = pd.DataFrame({
    "Subject_1": subject_1_list + subject_2_list,
    "Subject_2": subject_2_list + subject_1_list,
    "Clashes": Clashes + Clashes
})
print(df.head)

<bound method NDFrame.head of     Subject_1         Subject_2  Clashes
0       Maths           Physics        8
1       Maths         Chemistry        7
2       Maths        Philosophy        2
3       Maths     Further Maths       11
4       Maths            German        1
..        ...               ...      ...
85  Economics           Biology        1
86      OTHER           Biology        1
87  Economics  Computer Science        0
88      OTHER  Computer Science        2
89      OTHER         Economics        1

[90 rows x 3 columns]>


In [10]:
def optimize_timetable(df, n_slots):
    """
    Optimize a timetable by distributing subjects across n slots to minimize clashes.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns Subject_1, Subject_2, and Clashes
    n_slots (int): Number of time slots to distribute subjects into
    
    Returns:
    tuple: (min_score, slots) where min_score is the minimum clash count and 
           slots is a list of lists containing subjects assigned to each slot
    """
    # Extract unique subjects from the dataframe
    all_subjects = set()
    for col in ['Subject_1', 'Subject_2']:
        all_subjects.update(df[col].unique())
    subjects = list(all_subjects)
    num_subjects = len(subjects)
    # Verify we have enough subjects for the requested number of slots
    if num_subjects < n_slots:
        raise ValueError(f"Not enough subjects ({num_subjects}) to distribute across {n_slots} slots")
    
    # Create a helper function to calculate clashes for a set of subjects
    def calculate_clashes(subject_set):
        if len(subject_set) < 2:
            return 0
        
        pairs = list(combinations(subject_set, 2))
        clash_count = 0
        
        for pair in pairs:
            pair_clashes = df.loc[
                ((df['Subject_1'] == pair[0]) & (df['Subject_2'] == pair[1])) | 
                ((df['Subject_1'] == pair[1]) & (df['Subject_2'] == pair[0])),
                'Clashes'
            ]
            if not pair_clashes.empty:
                clash_count += pair_clashes.iloc[0]
                
        return clash_count
    
    # Initialize variables to track the best solution
    min_score = float('inf')
    best_slots = []
    
    # Recursive function to try different distributions of subjects
    def distribute_subjects(remaining_subjects, current_slots, depth):
        nonlocal min_score, best_slots
        
        # Base case: all slots filled except the last one
        if depth == n_slots - 1:
            # Assign all remaining subjects to the last slot
            trial_slots = current_slots + [remaining_subjects]
            
            # Calculate total clashes
            total_clashes = sum(calculate_clashes(slot) for slot in trial_slots)
            
            # Update if better
            if total_clashes < min_score:
                min_score = total_clashes
                best_slots = trial_slots.copy()
            return
        
        # Recursive case: try different assignments for current slot
        # Calculate minimum subjects needed per slot (to ensure all slots get at least 1 subject)
        min_subjects_per_slot = 1
        
        # Calculate maximum subjects allowed in current slot
        # This ensures we leave enough subjects for remaining slots
        max_subjects = len(remaining_subjects) - (n_slots - depth - 1) * min_subjects_per_slot
        
        # Try different numbers of subjects for the current slot
        for i in range(min_subjects_per_slot, max_subjects + 1):
            # Generate all combinations of i subjects from remaining_subjects
            for combo in combinations(remaining_subjects, i):
                combo_list = list(combo)
                # Subjects not selected for current slot
                next_remaining = [s for s in remaining_subjects if s not in combo_list]
                # Recursive call to fill next slot
                distribute_subjects(next_remaining, current_slots + [combo_list], depth + 1)
    
    # Start the recursive distribution process
    distribute_subjects(subjects, [], 0)
    
    return min_score, best_slots

In [11]:
if __name__ == "__main__":
        
  
    # Specified Number of slots
    min_clashes, slot_assignment = optimize_timetable(df, n_slots=NUMBER_OF_SLOTS)
    n= len(slot_assignment)
    print(f"\nWith {n} slots: {min_clashes} clashes")
    for i, slot in enumerate(slot_assignment):
        print(f"  Slot {i+1}: {slot}")


With 3 slots: 13 clashes
  Slot 1: ['Philosophy', 'Maths']
  Slot 2: ['OTHER', 'Further Maths', 'Biology']
  Slot 3: ['Chemistry', 'German', 'Economics', 'Computer Science', 'Physics']


In [ ]:
# Lowest number of slots to get under number of acceptable clashes
    
    
slots = 1
min_clashes = float('inf')  # Initialize to a large value

while min_clashes > ACCEPTABLE_CLASHES:
    
    if __name__ == "__main__":
        
  
        min_clashes, slot_assignment = optimize_timetable(df, n_slots=slots)
        n= len(slot_assignment)
        slots+=1
        print(slots)
    
print(f"\nWith {n} slots: {min_clashes} clashes")
for i, slot in enumerate(slot_assignment):
    print(f"  Slot {i+1}: {slot}")


In [ ]:
# Lowest number of slots to get under acceptable percentage of clashes

slots = 1
min_clashes = float('inf')  # Initialize to a large value
slot_assignment = None

while ((min_clashes/Total_exams)*100) > ACCEPTABLE_CLASH_PERCENTAGE:
    
    if __name__ == "__main__":
        
  
        min_clashes, slot_assignment = optimize_timetable(df, n_slots=slots)
        n= len(slot_assignment)
        slots+=1
    
clash_percentage = round((min_clashes/Total_exams)*100)    
print(f"\nWith {n} slots: {min_clashes} clashes where {clash_percentage}% of exams have a clash")
for i, slot in enumerate(slot_assignment):
    print(f"  Slot {i+1}: {slot}")